In [ ]:
import ast
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import community as community_louvain
from itertools import combinations

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [13]:
df = pd.read_csv('data/flu_data_token.csv')
df

,Year,Content,Token
0,2006-2007,When and where did the 2006-07 flu season star...,"['200607', 'flu', 'season', 'start', 'first', ..."
1,2007-2008,How severe was the 2007-2008 flu season?: A gr...,"['severe', '20072008', 'flu', 'season', 'great..."
2,2008-2009,What was the 2009-2010 flu season like?: Flu s...,"['20092010', 'flu', 'season', 'like', 'flu', '..."
3,2010-2011,What was the 2010-2011 flu season like?: In co...,"['20102011', 'flu', 'season', 'like', 'compari..."
4,2011-2012,What was the 2011-2012 flu season like?: In co...,"['20112012', 'flu', 'season', 'like', 'compari..."
5,2012-2013,What was the 2012-2013 flu season like?: In co...,"['20122013', 'flu', 'season', 'like', 'compari..."
6,2013-2014,When did flu activity peak?: The timing of flu...,"['flu', 'activity', 'peak', 'timing', 'flu', '..."
7,2014-2015,What was the 2014-2015 flu season like?: Compa...,"['20142015', 'flu', 'season', 'like', 'compare..."
8,2015-2016,Season Summary Reports: nan\n\nWhat was the 20...,"['season', 'summary', 'report', 'nan', '201520..."
9,2016-2017,Season Summary Reports: nan\n\nInformation for...,"['season', 'summary', 'report', 'nan', 'inform..."


In [16]:
token: list = df['Token'][0]

In [17]:
token_list = ast.literal_eval(token)

In [18]:
print(token_list)

['200607', 'flu', 'season', 'start', 'first', 'report', 'regional', 'flu', 'activity', 'come', 'southeastern', 'united', 'states', 'first', 'week', 'november', 'regional', 'flu', 'activity', 'define', 'increase', 'flulike', 'activity', 'flu', 'outbreak', 'least', 'two', 'half', 'region', 'state', 'recent', 'laboratory', 'evidence', 'flu', 'region', 'severe', '20062007', 'flu', 'season', '200607', 'flu', 'season', 'generally', 'mild', 'compare', 'recent', 'flu', 'season', 'example', 'proportion', 'death', 'associate', 'influenza', 'illness', 'low', 'season', 'previous', 'three', 'flu', 'season', 'hospitalization', 'rate', 'among', 'child', 'also', 'low', 'previous', 'three', 'flu', 'season', 'however', 'pediatric', 'death', 'relate', 'influenza', 'report', '200607', 'season', 'previous', 'two', 'season', 'nationally', 'low', 'level', 'flu', 'activity', 'report', 'october', 'middecember', 'flu', 'activity', 'increase', 'late', 'december', 'peak', 'midfebruary', 'decrease', 'end', 'flu', 

### Word count

In [19]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist


def word_count(cleaned_tokens):
  # Calculate word frequencies
  word_freq = FreqDist(cleaned_tokens)

  # Convert word frequencies to a DataFrame for seaborn
  data = {'Word': __builtins__.list(word_freq.keys()), 'Frequency': __builtins__.list(word_freq.values())}
  df_word_freq = pd.DataFrame(data)

  # Sort DataFrame by frequency in descending order
  df_word_freq = df_word_freq.sort_values(by='Frequency', ascending=False)
  print(df_word_freq.head(60))

  return df_word_freq

In [20]:
df_word_freq = word_count(token_list)

             Word  Frequency
1             flu         48
35      influenza         34
2          season         31
94          virus         26
33          death         19
43          child         16
182       vaccine         14
7        activity         12
68           year         11
0          200607         11
5          report         10
4           first          9
125           cdc          8
273         novel          8
119  fluassociate          6
88         number          6
253          dose          6
54           peak          6
206           age          5
179         match          5
42          among          5
190       lineage          5
32     proportion          5
22          state          5
141        system          5
45        however          5
108       percent          5
79           test          5
120    notifiable          5
166             b          5
160     circulate          5
121     condition          5
198         month          4
116        peo

### Word Cloud

In [ ]:
def token2word(token):
  word = ' '.join(token)
  return word

In [ ]:
word = token2word(token_list)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud


def wordcloud_vis(word):
  # Generate word cloud
  wordcloud = WordCloud(width=800, height=400, background_color='white').generate(word)

  # Display the generated word cloud using matplotlib
  plt.figure(figsize=(10, 5))
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis('off')
  plt.show()

In [ ]:
wordcloud_vis(word)

### Bar chart

In [ ]:
def bar_chart_vis(token):

  word_freq = FreqDist(token)
  # Convert word frequencies to a DataFrame for seaborn
  data = {'Word': __builtins__.list(word_freq.keys()), 'Frequency': __builtins__.list(word_freq.values())}
  df_word_freq = pd.DataFrame(data)

  # Sort DataFrame by frequency in descending order
  df_word_freq = df_word_freq.sort_values(by='Frequency', ascending=False)

  # Plot a bar chart using seaborn
  plt.figure(figsize=(12, 6))
  sns.barplot(x='Word', y='Frequency', data=df_word_freq.head(20), palette='viridis')
  plt.title('Top 20 Most Frequent Words')
  plt.xlabel('Words')
  plt.ylabel('Frequency')
  plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
  plt.show()

In [ ]:
bar_chart_vis(token_list)

### Bubble chart

In [ ]:
def bubble_chart_vis(cleaned_tokens):

  # Calculate word frequencies
  word_freq = FreqDist(cleaned_tokens)

  # Create a DataFrame with word frequencies and lengths
  data = {'Word': __builtins__.list(word_freq.keys()), 'Frequency': __builtins__.list(word_freq.values()), 'Length': [len(word) for word in word_freq.keys()]}
  df_word_data = pd.DataFrame(data)

  # Filter out words with frequency less than 2 for better visualization
  df_word_data = df_word_data[df_word_data['Frequency'] >= 2]

  # Plot a bubble chart using seaborn
  plt.figure(figsize=(12, 8))
  sns.scatterplot(x='Length', y='Frequency', size='Frequency', data=df_word_data, hue='Word', sizes=(50, 300), palette='viridis', alpha=0.8)
  plt.title('Bubble Chart of Word Frequencies and Lengths')
  plt.xlabel('Word Length')
  plt.ylabel('Frequency')
  plt.show()

In [ ]:
bubble_chart_vis(token_list)

### Network Diagram

In [ ]:
def network_vis(df_word_freq):
    token_rank = df_word_freq.head(20)
    tr = token_rank['Word'].tolist()

    # Create a graph using networkx
    G = nx.Graph()

    # Create edges between co-occurring words
    for word1, word2 in combinations(tr, 2):
        if G.has_edge(word1, word2):
            G[word1][word2]['weight'] += 1
        else:
            G.add_edge(word1, word2, weight=1)

    # Detect communities (clusters) in the graph
    partition = community_louvain.best_partition(G)
    # Map each community to a distinct color
    community_colors = [partition[n] for n in G.nodes()]

    # Set node size based on degree (number of connections)
    node_size = [deg * 100 for deg in dict(G.degree()).values()]  # Adjusted size for better visibility

    # Draw the network diagram with community-based coloring
    plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(G, seed=42)  # For consistent layout
    nx.draw(G, pos, with_labels=True, font_size=8, node_size=node_size,
            cmap=plt.cm.jet, node_color=community_colors, font_color='white',
            edge_color='gray', font_weight='bold', alpha=0.7)
    plt.title('Text Network Diagram based on Word Co-occurrence with Clustering')
    plt.show()

In [ ]:
network_vis(df_word_freq)